## Setup

In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
customer_data = pd.read_csv("../data/raw/eci_customer_data.csv")
product_groups_data = pd.read_csv("../data/raw/eci_product_groups.csv")
product_master_data = pd.read_csv("../data/raw/eci_product_master.csv")
stores_clusters_data = pd.read_csv("../data/raw/eci_stores_clusters.csv")
stores_data = pd.read_csv("../data/raw/eci_stores.csv")
transactions_data = pd.read_csv("../data/raw/eci_transactions.csv")

## Exploracion de datos / Data imputation

### Transactions

In [4]:
transactions_data.head()

,TRANSACTION_ID,DATE,STORE_ID,SKU,QUANTITY,PRICE,TOTAL_SALES,SUBGROUP,STORE_SUBGROUP_DATE_ID
0,17185,2021-01-01,S00148,CLOMEJA001,1.0,52.80,52.80,Jackets,S00148_Jackets_2021-01-01
1,4108,2021-01-01,S00035,CLOCHIN014,1.0,55.25,55.25,Infants,S00035_Infants_2021-01-01
2,2400,2021-01-01,S00020,BEAHASH005,1.0,40.19,40.19,Shampoo,S00020_Shampoo_2021-01-01
3,7978,2021-01-01,S00068,TOYGABO004,1.0,35.41,35.41,Board Games,S00068_Board Games_2021-01-01
4,6946,2021-01-01,S00059,BEAHASH006,1.0,38.28,38.28,Shampoo,S00059_Shampoo_2021-01-01


In [5]:
# Convertimos la columna DATE a tipo datetime
transactions_data["DATE"] = pd.to_datetime(transactions_data["DATE"], format="%Y-%m-%d")

In [ ]:
# Tenemos data desde 2021-01-01 hasta 2023-12-31 (3 años)
transactions_data.describe().round(2)

,TRANSACTION_ID,DATE,QUANTITY,PRICE,TOTAL_SALES
count,19004759.00,19004759,18909672.00,19004759.00,19004759.00
mean,9502380.00,2022-06-22 20:26:04.793325568,1.44,68.69,97.37
min,1.00,2021-01-01 00:00:00,1.00,4.02,4.02
25%,4751190.50,2021-09-23 00:00:00,1.00,22.94,25.50
50%,9502380.00,2022-06-21 00:00:00,1.00,41.24,48.01
75%,14253569.50,2023-03-16 00:00:00,2.00,74.02,93.02
max,19004759.00,2023-12-31 00:00:00,85.00,431.87,8369.26
std,5486201.51,NaN,1.01,84.06,155.90


In [ ]:
transactions_data.isna().sum()

TRANSACTION_ID                0
DATE                          0
STORE_ID                      0
SKU                           0
QUANTITY                  95087
PRICE                         0
TOTAL_SALES                   0
SUBGROUP                      0
STORE_SUBGROUP_DATE_ID        0
dtype: int64


In [8]:
# Si la columna de Quantity es NaN, llenamos los valores de Quantitiy como Total_sales / Price
transactions_data['QUANTITY'] = transactions_data.apply(
    lambda row: row['TOTAL_SALES'] / row['PRICE'] if pd.isna(row['QUANTITY']) else row['QUANTITY'], axis=1
)

### Clientes

In [14]:
customer_data.head()

,client_id,phone_number,email_address,city,state,zip_code,education_level,occupation,loyalty_member,loyalty_number,loyalty_points
0,500001,NaN,n.edwards+spam@outlook.com,Spokane,WA,99201,High School,NaN,No,NaN,339
1,500002,(412) 698-7960,lisa_cook@yahoo.com,Pittsburgh,PA,15222,High School,Sales Representative,Yes,LP525082,406
2,500003,(303) 491-3774,NaN,Fort Collins,CO,80526,College Graduate,Analyst,True,950139,14
3,500004,(803) 255-0104,betty-campbell@gmail.com,Charleston,SC,29401,Some College,Supervisor,No,NaN,NaN
4,500005,(303) 596-8987,NaN,Aurora,CO,80012,Associates,NaN,Yes,NaN,NaN


In [15]:
# Cantidad de clientes
customer_data["client_id"].nunique()

801923

### Productos

In [9]:
product_groups_data.head()

,sku,product_name,price_group_id,price_group_name,group_type
0,BOOEDTE001,PageTurn Essential Textbooks,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
1,BOOEDTE002,Bookworm Advanced Textbooks,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
2,BOOEDTE004,PageTurn Essential Textbooks,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
3,CLOMESH002,DressRight Shirts Collection,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
4,CLOMESH005,StyleX Relaxed Shirts,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal


In [10]:
product_groups_data.isna().sum()

sku                                   0
 product_name                         0
 price_group_id                       0
 price_group_name                     0
 group_type                           0
dtype: int64

In [11]:
# Renombro columnas para que coincidan con las de transactions_data
product_master_data.rename(columns={"sku": "SKU"}, inplace=True)
product_groups_data.rename(columns={"sku       ": "SKU"}, inplace=True)

In [12]:
len(product_groups_data)

80

In [13]:
product_groups_data["SKU"].nunique()

73

In [14]:
# Hay repetidos, que se deben a que un producto puede pertenecer a varios grupos
product_groups_data[product_groups_data["SKU"].duplicated(keep=False)].sort_values("SKU")

,SKU,product_name,price_group_id,price_group_name,group_type
7,ELECOLA006,Electra Laptops Edge XL,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
31,ELECOLA006,Electra Laptops Edge XL,COMP_SETUP_01,Computer Setup Essentials,Bundle
51,ELECOLA006,Electra Laptops Edge XL,MOBILE_DEVICES_01,Personal Computing Devices,Substitute
8,ELECOLA009,ByteWare Laptops Smart 2025,BACK_TO_SCHOOL_01,Back to School Essentials,Seasonal
52,ELECOLA009,ByteWare Laptops Smart 2025,MOBILE_DEVICES_01,Personal Computing Devices,Substitute
34,ELEMOAC008,ByteWare Accessories Air 2025,COMP_SETUP_01,Computer Setup Essentials,Bundle
48,ELEMOAC008,ByteWare Accessories Air 2025,GIFT_UNDER_50_01,Gift Ideas Under $50,Promotional
35,ELEMOAC009,TechPro Accessories Air X,COMP_SETUP_01,Computer Setup Essentials,Bundle
49,ELEMOAC009,TechPro Accessories Air X,GIFT_UNDER_50_01,Gift Ideas Under $50,Promotional
38,SPOFIEQ004,FitZone Sport Equipment,FITNESS_COMBO_01,Complete Fitness Package,Bundle


In [15]:
# Dado que no se diferencian en las transacciones, los eliminamos por simplicidad
product_groups_data.drop_duplicates(subset=["SKU"], inplace=True)

In [16]:
product_master_data.head()

,SKU,product_name,category,group,subgroup,brand,base_price,initial_ticket_price,costos
0,ELECOLA001,TechPro Laptops Smart 2025,Electronics,Computing,Laptops,TechPro,284.94,355.14,225.84
1,ELECOLA002,Electra Laptops Ultra Pro,Electronics,Computing,Laptops,Electra,311.83,352.74,209.99
2,ELECOLA003,CircuitCity Laptops Air 2024,Electronics,Computing,Laptops,CircuitCity,246.96,301.35,192.61
3,ELECOLA004,TechPro Laptops Elite 2025,Electronics,Computing,Laptops,TechPro,324.96,420.49,212.23
4,ELECOLA005,CircuitCity Laptops Pro S,Electronics,Computing,Laptops,CircuitCity,339.88,386.23,231.75


In [17]:
product_master_data.isna().sum()

SKU                     0
product_name            0
category                0
group                   0
subgroup                0
brand                   0
base_price              0
initial_ticket_price    0
costos                  0
dtype: int64

In [18]:
# Notar el formato de las columnas de product_groups_data
product_groups_data.columns

Index(['SKU', ' product_name                     ', ' price_group_id    ',
       ' price_group_name          ', ' group_type'],
      dtype='object')

In [ ]:
# Unimos los datos de productos con los grupos de productos
productos = pd.merge(product_master_data, product_groups_data, on="SKU", how="left")

In [20]:
# Hay muchos productos que no tienen grupo asignado
productos.isna().sum()

SKU                                     0
product_name                            0
category                                0
group                                   0
subgroup                                0
brand                                   0
base_price                              0
initial_ticket_price                    0
costos                                  0
 product_name                         788
 price_group_id                       788
 price_group_name                     788
 group_type                           788
dtype: int64

In [21]:
# Asignamos Comun a los productos que no tienen grupo asignado (price_group_id y group_type, el resto no las utilizamos)
productos[" price_group_id    "] = productos[' price_group_id    '].fillna('Comun')
productos[' group_type']= productos[' group_type'].fillna('Comun')


### Stores

In [22]:
stores_clusters_data.head()

,STORE_ID,BRAND,STORE_NAME,CLUSTER
0,S00001,PrimeGoods,PrimeGoods - Hillsboro,Cluster_West_Coast
1,S00002,MegaShop,MegaShop Indianapolis Eastside,Midwest_Cluster
2,STORE0003,MarketExpress,MarketExpress - Las Vegas,Mountain_Cluster
3,STORE0004,MegaShop,MegaShop - Oklahoma City,Texas_Cluster
4,S00005,PrimeGoods,PrimeGoods - Charlotte,NC_Cluster


In [31]:
stores_clusters_data.describe()

,STORE_ID,BRAND,STORE_NAME,CLUSTER
count,157,157,157,140
unique,157,9,144,13
top,S00001,EssentialsPlus,EssentialsPlus - Spartanburg,Cluster_West_Coast
freq,1,25,3,16


In [23]:
# Aunque Cluster tenga NaN, no es necesario el dato dado que ya sabemos el estado y la ciudad de cada tienda
stores_clusters_data.isna().sum()

STORE_ID       0
BRAND          0
STORE_NAME     0
CLUSTER       17
dtype: int64

In [31]:
# Notar que hay diferentes formatos de STORE_ID, pero coinciden con los de stores_data y los de transactions_data
(stores_clusters_data["STORE_ID"].unique() == stores_data["STORE_ID"].unique()).sum()

np.int64(157)

In [33]:
stores_data.head()

,STORE_ID,BRAND,STORE_NAME,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,OPENDATE,CLOSEDATE,STORE_TYPE,REGION
0,S00001,PrimeGoods,PrimeGoods - Hillsboro,5735 NE Washington Pl,NaN,Hillsboro,OR,74820,2015-01-10 00:00:00,NaN,Express,West
1,S00002,MegaShop,MegaShop Indianapolis Eastside,3557 Cedar Ct,Unit 89,Indianapolis,IN,33483-1775,2012-03-23 00:00:00,NaN,Express,Midwest
2,STORE0003,MarketExpress,MarketExpress - Las Vegas,7870 W Church Ave,NaN,Las Vegas,NV,49099,2021-09-23 00:00:00,NaN,Outlet,West
3,STORE0004,MegaShop,MegaShop - Oklahoma City,3105 N Washington Pl,NaN,Oklahoma City,OK,80592,06/30/2015,NaN,Mall,Southwest
4,S00005,PrimeGoods,PrimeGoods - Charlotte,8756 SW Maple Ln,Suite 848,Charlotte,NC,32299,2019-06-10 00:00:00,NaN,Express,Southeast


In [35]:
stores_data.describe()

,STORE_ID,BRAND,STORE_NAME,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,OPENDATE,CLOSEDATE,STORE_TYPE,REGION
count,157,157,157,157,43,157,157,151,157,18,155,157
unique,157,9,144,157,34,77,22,151,155,18,8,5
top,S00001,EssentialsPlus,EssentialsPlus - Spartanburg,5735 NE Washington Pl,Building E,Hillsboro,SC,74820,2017-09-27 00:00:00,2028-02-21 00:00:00,Mall,Southeast
freq,1,25,3,1,3,5,13,1,2,1,44,47


In [34]:
stores_data.isna().sum()

STORE_ID        0
BRAND           0
STORE_NAME      0
ADDRESS1        0
ADDRESS2      114
CITY            0
STATE           0
ZIP             6
OPENDATE        0
CLOSEDATE     139
STORE_TYPE      2
REGION          0
dtype: int64

In [24]:
# Completamos los NaN en STORE_TYPE con el valor mas frecuente
stores_data["STORE_TYPE"]= stores_data["STORE_TYPE"].fillna(stores_data["STORE_TYPE"].mode()[0])

In [27]:
# Hay distintos formatos de fechas, lo unificamos
stores_data_fixed = stores_data.copy()
stores_data_fixed["OPENDATE"] = pd.to_datetime(stores_data["OPENDATE"], errors='coerce')
mask = stores_data_fixed[stores_data_fixed["OPENDATE"].isna()]
stores_data_fixed.loc[mask.index, "OPENDATE"] = pd.to_datetime(stores_data.loc[mask.index, "OPENDATE"], errors='coerce')

In [28]:
# Reemplazamos los valores de CLOSDATE que son NaN o mayor que la fecha actual, por la fecha actual más un año (indicando que todavia no cerro)
stores_data_fixed["CLOSEDATE"] = pd.to_datetime(stores_data["CLOSEDATE"], errors='coerce')

# Fecha de hoy
fecha_hoy = pd.to_datetime("today", format='%Y-%m-%d').normalize()
stores_data_fixed.loc[
    (stores_data_fixed["CLOSEDATE"].isna()) | 
    (stores_data_fixed["CLOSEDATE"] > fecha_hoy), 
    "CLOSEDATE"
] = fecha_hoy + pd.DateOffset(years=1)

In [29]:
stores_data_fixed.head()

,STORE_ID,BRAND,STORE_NAME,ADDRESS1,ADDRESS2,CITY,STATE,ZIP,OPENDATE,CLOSEDATE,STORE_TYPE,REGION
0,S00001,PrimeGoods,PrimeGoods - Hillsboro,5735 NE Washington Pl,NaN,Hillsboro,OR,74820,2015-01-10,2026-07-28,Express,West
1,S00002,MegaShop,MegaShop Indianapolis Eastside,3557 Cedar Ct,Unit 89,Indianapolis,IN,33483-1775,2012-03-23,2026-07-28,Express,Midwest
2,STORE0003,MarketExpress,MarketExpress - Las Vegas,7870 W Church Ave,NaN,Las Vegas,NV,49099,2021-09-23,2026-07-28,Outlet,West
3,STORE0004,MegaShop,MegaShop - Oklahoma City,3105 N Washington Pl,NaN,Oklahoma City,OK,80592,2015-06-30,2026-07-28,Mall,Southwest
4,S00005,PrimeGoods,PrimeGoods - Charlotte,8756 SW Maple Ln,Suite 848,Charlotte,NC,32299,2019-06-10,2026-07-28,Express,Southeast


In [30]:
# Como no utilizamos Adress, podemos dejar que haya NaN
stores_data_fixed.isna().sum()

STORE_ID        0
BRAND           0
STORE_NAME      0
ADDRESS1        0
ADDRESS2      114
CITY            0
STATE           0
ZIP             6
OPENDATE        0
CLOSEDATE       0
STORE_TYPE      0
REGION          0
dtype: int64

## Merge de datos

In [33]:
stores = pd.merge(stores_data_fixed, stores_clusters_data, on="STORE_ID", how="left")

In [34]:
datos_unidos = pd.merge(transactions_data, stores, on="STORE_ID", how="left")

In [35]:
datos_unidos = pd.merge(datos_unidos, productos, on="SKU", how="left")

In [36]:
datos_unidos.columns

Index(['TRANSACTION_ID', 'DATE', 'STORE_ID', 'SKU', 'QUANTITY', 'PRICE',
       'TOTAL_SALES', 'SUBGROUP', 'STORE_SUBGROUP_DATE_ID', 'BRAND_x',
       'STORE_NAME_x', 'ADDRESS1', 'ADDRESS2', 'CITY', 'STATE', 'ZIP',
       'OPENDATE', 'CLOSEDATE', 'STORE_TYPE', 'REGION', 'BRAND_y',
       'STORE_NAME_y', 'CLUSTER', 'product_name', 'category', 'group',
       'subgroup', 'brand', 'base_price', 'initial_ticket_price', 'costos',
       ' product_name                     ', ' price_group_id    ',
       ' price_group_name          ', ' group_type'],
      dtype='object')

In [37]:
# Renombramos las columnas
datos_unidos.rename(columns={" price_group_id    ": "PRICE_GROUP_ID", "BRAND_x": "BRAND",
                             "category":"CATEGORY", "group":"GROUP", "base_price": "BASE_PRICE", 
                             "initial_ticket_price": "INITIAL_TICKET_PRICE", "costos": "COSTOS",
                             " group_type": "GROUP_TYPE"}, inplace=True)

In [43]:
# Nos quedamos con las columnas que nos interesan
# Notar que dejamos fuera TOTAL_SALES, ya que es redundante con PRICE * QUANTITY
datos_unidos = datos_unidos[["TRANSACTION_ID" , "DATE",  "SKU", "QUANTITY", "PRICE",
    "REGION", "CITY", "STATE", "STORE_TYPE", "STORE_ID", "OPENDATE", "CLOSEDATE", "STORE_SUBGROUP_DATE_ID",
    "CATEGORY", "GROUP", "SUBGROUP","GROUP_TYPE", "PRICE_GROUP_ID", "BRAND", "INITIAL_TICKET_PRICE", "BASE_PRICE", "COSTOS" ]]

In [39]:
len(datos_unidos)

19159425

In [44]:
# No aseguramos que no haya valores NaN
datos_unidos.isna().sum()

TRANSACTION_ID            0
DATE                      0
SKU                       0
QUANTITY                  0
PRICE                     0
REGION                    0
CITY                      0
STATE                     0
STORE_TYPE                0
STORE_ID                  0
OPENDATE                  0
CLOSEDATE                 0
STORE_SUBGROUP_DATE_ID    0
CATEGORY                  0
GROUP                     0
SUBGROUP                  0
GROUP_TYPE                0
PRICE_GROUP_ID            0
BRAND                     0
INITIAL_TICKET_PRICE      0
BASE_PRICE                0
COSTOS                    0
dtype: int64

## Feature Engineering

In [54]:
# Nos aseguramos que este ordenado por fecha
datos_unidos.sort_values(by="DATE", inplace=True)

In [ ]:
datos_unidos["YEAR"] = datos_unidos["DATE"].dt.year
datos_unidos["MONTH"] = datos_unidos["DATE"].dt.month
datos_unidos["DAY"] = datos_unidos["DATE"].dt.day
datos_unidos["DAY_OF_WEEK"] = datos_unidos["DATE"].dt.day_name()
datos_unidos["WEEK"] = datos_unidos["DATE"].dt.isocalendar().week

In [180]:
def rolling_features(df, group_col, windows = [7,30, 90, 180]):
    ventas_diarias = df.groupby([group_col, 'DATE']).agg(
        total_cantidad_diaria=('QUANTITY', 'sum')).reset_index()
    
    ventas_diarias = ventas_diarias.sort_values(by='DATE')
    
    for window in windows:
        ventas_diarias[f"{group_col}_mean_{window}D"] = ventas_diarias.groupby(group_col)["total_cantidad_diaria"].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)
        ventas_diarias[f"{group_col}_mean_{window}D"] = ventas_diarias.groupby(group_col)["total_cantidad_diaria"].rolling(window=window, min_periods=1).std().reset_index(level=0, drop=True)

    df = pd.merge(
        df,
        ventas_diarias[[group_col, 'DATE'] + [f"{group_col}_mean_{window}D" for window in windows]],
        on=[group_col, 'DATE'],
        how='left'
    )

    return df

In [181]:
datos_unidos = rolling_features(datos_unidos, 'SKU')

In [ ]:
datos_unidos = rolling_features(datos_unidos, 'STORE_ID')

In [ ]:
# Guardamos el DataFrame procesado
datos_unidos.to_csv("../data/procesados/datos_unidos.csv", index=False)

## Outliers

In [75]:
datos_unidos.describe().round(2)

,TRANSACTION_ID,QUANTITY,PRICE,TOTAL_SALES,OPENDATE,CLOSEDATE,INITIAL_TICKET_PRICE,BASE_PRICE,COSTOS
count,19159425.00,19159425.00,19159425.00,19159425.00,19159425,19159425,19159425.00,19159425.00,19159425.00
mean,9502506.90,1.43,68.84,97.58,2015-08-13 13:58:48.432632832,2026-07-13 15:31:23.542142464,84.20,70.23,44.10
min,1.00,1.00,4.02,4.02,2001-08-29 00:00:00,2023-11-01 00:00:00,8.88,8.02,5.76
25%,4751389.00,1.00,23.07,25.64,2012-05-29 00:00:00,2026-07-28 00:00:00,28.32,24.03,11.16
50%,9502476.00,1.00,41.57,48.39,2016-02-26 00:00:00,2026-07-28 00:00:00,50.12,42.58,19.70
75%,14253732.00,2.00,75.16,93.48,2019-10-14 00:00:00,2026-07-28 00:00:00,93.26,77.90,45.74
max,19004759.00,85.00,431.87,8369.26,2023-10-14 00:00:00,2026-07-28 00:00:00,452.69,358.79,281.26
std,5486267.37,1.00,83.74,155.44,NaN,NaN,101.88,84.80,63.15


In [78]:
datos_unidos.QUANTITY.quantile(0.999)

np.float64(10.0)

In [79]:
# Eliminamos los outliers de la columna QUANTITY, aquellos que superen el 99.9%
datos_unidos = datos_unidos[datos_unidos.QUANTITY <= datos_unidos.QUANTITY.quantile(0.999)]

In [80]:
datos_unidos.describe().round(2)

,TRANSACTION_ID,QUANTITY,PRICE,TOTAL_SALES,OPENDATE,CLOSEDATE,INITIAL_TICKET_PRICE,BASE_PRICE,COSTOS
count,19142791.00,19142791.00,19142791.00,19142791.00,19142791,19142791,19142791.00,19142791.00,19142791.00
mean,9503535.76,1.42,68.84,96.90,2015-08-13 13:31:45.718787072,2026-07-13 15:27:49.315259648,84.19,70.22,44.10
min,1.00,1.00,4.02,4.02,2001-08-29 00:00:00,2023-11-01 00:00:00,8.88,8.02,5.76
25%,4752557.50,1.00,23.07,25.63,2012-05-29 00:00:00,2026-07-28 00:00:00,28.32,24.03,11.16
50%,9504335.00,1.00,41.57,48.35,2016-02-26 00:00:00,2026-07-28 00:00:00,50.12,42.58,19.70
75%,14255741.50,2.00,75.16,93.34,2019-10-14 00:00:00,2026-07-28 00:00:00,93.26,77.90,45.74
max,19004759.00,10.00,431.87,4010.50,2023-10-14 00:00:00,2026-07-28 00:00:00,452.69,358.79,281.26
std,5486575.39,0.93,83.75,150.91,NaN,NaN,101.88,84.80,63.14
